In [ ]:
from mpcontribs.client import load_client
from mpcontribs.io.core.recdict import RecursiveDict

In [ ]:
client = load_client() # add api key from https://portal.mpcontribs.org/dashboard as argument

## project info & overview

In [ ]:
project = 'carrier_transport'
info = client.projects.get_entry(project=project).response().result
RecursiveDict(info)

## specific contribution

In [ ]:
# use https://portal.mpcontribs.org to find material with contribution for specific project
project, mpid = 'carrier_transport', 'mp-27502'
cid = mpr.query_contributions(criteria={'mp_cat_id': mpid})[0]['_id']
mpfile = mpr.find_contribution(cid)
print 'see https://portal.mpcontribs.org/explorer/{}'.format(portal, cid)

### hierarchical data

In [ ]:
mpfile.hdata[mpid]['data'] # dictionary

### tabular data

In [ ]:
df = mpfile.tdata[mpid]['S(p)'] # Pandas DataFrame
df.head()

### graphs

In [ ]:
mpfile.gdata[mpid]['σ(p)']

## query

In [ ]:
# direct querying of values will be possible once MPContribs has switched to SciData internally (better units handling)
docs = mpr.query_contributions(projection={'content.data.σ.p.<ε>': 1, 'mp_cat_id': 1})
print 'found {} contributions'.format(len(docs))

In [ ]:
observable, doping, threshold = 'σ', 'p', 1e16
results = {}
for doc in docs:
    identifier = doc['mp_cat_id']
    v = doc['content']['data'][observable][doping]['<ε>'].split() # split value and unit (see note above)
    if float(v[0]) > threshold:
        results[identifier] = float(v[0])
print 'found {} materials with {}({}) > {} {}'.format(len(results), observable, doping, threshold, v[1])